# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome()
browser.get('https://www.google.com')
browser.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
browser.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = browser.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
browser.get('https://www.google.com')
browser.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
browser.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = browser.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
browser.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = browser.find_element('xpath', '//*[@id="comercial"]').get_attribute('value').replace(',', '.')

browser.quit()
print(cotacao_dolar, cotacao_euro, cotacao_ouro)

5.096 5.5283956000000005 315.12


### Atualizando a base de preços com as novas cotações

In [2]:
import pandas as pd

df = pd.read_excel('Produtos.xlsx')
display(df)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.0832,5083.149168,1.40,7116.408835
1,Carro Renault,4500.00,Euro,5.5154,24819.300000,2.00,49638.600000
2,Notebook Dell,899.99,Dólar,5.0832,4574.829168,1.70,7777.209586
3,IPhone,799.00,Dólar,5.0832,4061.476800,1.70,6904.510560
4,Carro Fiat,3000.00,Euro,5.5154,16546.200000,1.90,31437.780000
5,Celular Xiaomi,480.48,Dólar,5.0832,2442.375936,2.00,4884.751872
6,Joia 20g,20.00,Ouro,310.7600,6215.200000,1.15,7147.480000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
df.loc[df['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
df.loc[df['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
df.loc[df['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

df['Preço de Compra'] = df['Cotação'] * df['Preço Original']
df['Preço de Venda'] = df['Preço de Compra'] * df['Margem']
display(df)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.096000,5095.94904,1.40,7134.328656
1,Carro Renault,4500.00,Euro,5.528396,24877.78020,2.00,49755.560400
2,Notebook Dell,899.99,Dólar,5.096000,4586.34904,1.70,7796.793368
3,IPhone,799.00,Dólar,5.096000,4071.70400,1.70,6921.896800
4,Carro Fiat,3000.00,Euro,5.528396,16585.18680,1.90,31511.854920
5,Celular Xiaomi,480.48,Dólar,5.096000,2448.52608,2.00,4897.052160
6,Joia 20g,20.00,Ouro,315.120000,6302.40000,1.15,7247.760000


### Exportando a nova base de preços atualizada

In [4]:
df.to_excel('Produtos.xlsx', index=False)